# The barotropic vorticity equation

The barotropic vorticity equation (BVE) is given by

$$\frac{\partial \zeta}{\partial t}+\mathbf u \nabla\zeta=0$$

It can be interpreted as describing the 2D advection of vorticity $\zeta$. The advection speed is the geostrophic wind, which is given by

$$\mathbf u=\mathbf k \times \nabla \psi,$$

where $\mathbf k$ is the vertical unit vector, and $\psi$ is the streamfunction, which is related to the vorticity as

$$\zeta=\nabla^2\psi.$$

This means that the advection speed depends on the vorticity, making this a *nonlinear* problem.


## Libraries and auxiliary functions

First, numpy and matplotlib are loaded, and a function is defined to create animations.

In [ ]:
## Load numpy and matplotlib libraries
import numpy as np
%matplotlib notebook
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.ioff()
plt.rcParams["animation.html"] = "jshtml"
plt.rcParams['figure.dpi'] = 100  # reduce to generate figures faster (but smaller)

# function for animating results
def create_animation(psi,frames=None,levels=None):
    
    # create figure with axes
    fig,ax=plt.subplots()

    # determine frames to plot
    if frames is None:
        # number of timesteps
        nt=psi.shape[0]-1
        frames=np.arange(0,nt+1)
    else:
        nt=frames[-1]
    
    # determine levels
    if levels is None:
        # generate nice levels between min and max of psi
        nLevels=20
        zmin=1.2*np.nanmin(psi[0])-0.2*np.nanmax(psi[0])
        zmax=1.2*np.nanmax(psi[0])-0.2*np.nanmin(psi[0])
        s=(zmax-zmin)/nLevels  # first guess
        ss=0.7*s/10**np.floor(np.log10(s))  # between 1 and 10
        if ss<2:
            ss=2
        elif ss<5:
            ss=5
        else:
            ss=10
        s=ss*10**np.floor(np.log10(s))
        levels=s*np.arange(np.floor(zmin/s),np.ceil(zmax/s)+1)

        
    h = ax.axis([x[0],x[-1],y[0],y[-1]])
    
    # define plot function for a single time step
    def plotResults(it):
        ax.clear()
        ax.set_title('%3i (%f)'%(it,t[it]))
        cs=ax.contour(xg,yg,psi[it],levels)
        ax.clabel(cs, cs.levels, inline=True);
    
    # animate over timesteps
    ani = animation.FuncAnimation(fig, plotResults, frames=frames,cache_frame_data=False,blit=True)
    display(ani)

## Solving the BVE

The code below solves the BVE with a leapfrog time scheme and centered 2nd order finite differences.


In [ ]:
# parameters
nt=500
dt=1.0
nx=72
dx=1.0
ny=64
dy=1.0

# coordinates
x=np.arange(nx)*dx
y=np.arange(ny)*dy
t=np.arange(nt+1)*dt

# grid
xg,yg=np.meshgrid(x,y)

# allocate solution (stream function)
psi=np.zeros((nt+1,ny,nx))

# grid indices
jx=np.arange(nx)
jxL=(jx-1)%nx
jxR=(jx+1)%nx
jy=np.arange(ny)
jyL=(jy-1)%ny
jyR=(jy+1)%ny

# wavenumbers
kx=(np.arange(nx)+nx//2)%nx-nx//2; kx=2*np.pi/(nx*dx)*kx
ky=(np.arange(ny)+ny//2)%ny-ny//2; ky=2*np.pi/(ny*dy)*ky
kxg,kyg=np.meshgrid(kx,ky)

# initial states

# gaussian bell + harmonic function
#psi[0]=5*(np.exp(-40*( (xg/(nx*dx)-0.5)**2+(yg/(ny*dy)-0.5)**2 )) + np.sin(2*np.pi*xg/(nx*dx)))

# gaussian bell
#psi[0]=np.exp(-40*( (xg/(nx*dx)-0.5)**2+(yg/(ny*dy)-0.5)**2 ))

# single harmonic function
#psi[0]=np.cos(2*np.pi*xg/(nx*dx))*np.cos(4*np.pi*yg/(ny*dy))

# random field
psi[0]=nx*ny*np.real(np.fft.ifft2(
    np.exp(-30*(kxg**2+kyg**2))
    *np.exp(1j*2*np.pi*np.random.rand(ny,nx))
))

# Jacobian function of psi and zeta
def jacobian(psi,zeta):
    # calculate d(psi)/dx*d(zeta)/dy-d(psi)/dy*d(zeta)/dx
    J=((psi[:,jxR]-psi[:,jxL])*(zeta[jyR,:]-zeta[jyL,:])-(psi[jyR,:]-psi[jyL,:])*(zeta[:,jxR]-zeta[:,jxL]))/(4*dx*dy)
    return(J)

# Laplacian operator
def laplacian(psi):
    zeta=(psi[:,jxL]-2*psi[:,jx]+psi[:,jxR])/dx**2+(psi[jyL,:]-2*psi[jy,:]+psi[jyR,:])/dy**2
    return(zeta)

# inverse Laplacian operator (with FFT)
def invlaplacian(zeta):
    ZS=np.fft.fft2(zeta)
    ZS=-ZS/np.maximum((np.sin(kxg*dx/2)/(dx/2))**2+(np.sin(kyg*dy/2)/(dy/2))**2,1.e-16)
    ZS[(kxg==0) & (kyg==0)]=0
    psi=np.real(np.fft.ifft2(ZS))
    return(psi)

def maxwind(psi):
    U=np.sqrt( ((psi[jyR,:]-psi[jyL,:])/(2*dy))**2+((psi[:,jxR]-psi[:,jxL])/(2*dx))**2 )
    return(np.max(U))
    
# first timestep
jt=0
print('timestep ',jt,': max windspeed = %6.3f'%maxwind(psi[jt]))
zeta=laplacian(psi[jt])
dzetadt=-jacobian(psi[jt],zeta)
dpsidt=invlaplacian(dzetadt)
psi[jt+1]=psi[jt]+dt*dpsidt

# later timesteps
for jt in range(1,nt):
    zeta=laplacian(psi[jt])
    dzetadt=-jacobian(psi[jt],zeta)
    dpsidt=invlaplacian(dzetadt)
    psi[jt+1]=psi[jt-1]+2*dt*dpsidt
    
# print final wind speed
jt=nt
print('timestep ',jt,': max windspeed = %6.3f'%maxwind(psi[jt]))

In [ ]:
create_animation(psi,frames=np.arange(0,nt+1,10))

In [ ]:
plt.figure()
plt.imshow(psi[0])
plt.colorbar()
plt.show()

In [ ]:
plt.figure()
z=(xg-nx*dx/2)**2+(yg-ny*dy/3)**2
z=z-np.mean(z)
z=laplacian(z)
dz=invlaplacian(laplacian(z))-z
plt.imshow(dz)
plt.colorbar()
plt.show()

In [ ]:
np.max(kx)